In [ ]:
from aocd import data, submit
from aocd.models import Puzzle

In [ ]:
import itertools
import functools
import random
import collections
import operator
import sys
import re
import copy
import numpy as np
import math
import string
import networkx as nx

In [ ]:
from helpers.functions import *

Configuration

In [ ]:
DIR = "data/2020/"
load_day = functools.partial(load, DIR)

# Problems

## Day 1

http://adventofcode.com/2020/day/1

In [ ]:
content = mapl(int, load_day(1))

In [ ]:
next(a*b for a, b in itertools.combinations(content, r=2) if a + b == 2020)

In [ ]:
next(a*b*c for a, b, c in itertools.combinations(content, r=3) if a + b + c == 2020)

## Day 2

http://adventofcode.com/2020/day/2

In [ ]:
content = mapl(str, load_day(2))

In [ ]:
total = 0
for password in content:
    s, e, c, pswd = re.match("(\d+)-(\d+) (\w): (\w+)", password).groups()
    if int(s) <= pswd.count(c) <= int(e):
        total += 1
        
total

In [ ]:
total = 0
for password in content:
    s, e, c, pswd = re.match("(\d+)-(\d+) (\w): (\w+)", password).groups()
    if (pswd[int(s)-1] == c) ^ (pswd[int(e)-1] == c):
        total += 1
        
total

## Day 3

http://adventofcode.com/2020/day/3

In [ ]:
content = Puzzle(year=2020, day=3).input_data.split("\n")

In [ ]:
def get_tree(right, down):
    offset = count = 0
    for row in content[::down]:
        count += row[offset % len(content[0])] == "#"
        offset += right
    return count

In [ ]:
solution = get_tree(3, 1)
#submit(solution, part="a", day=3, year=2020)

In [ ]:
solution = get_tree(1, 1) * get_tree(3, 1) * get_tree(5, 1) * get_tree(7, 1) * get_tree(1, 2)
#submit(solution, part="b", day=3, year=2020)

## Day 4

http://adventofcode.com/2020/day/4

In [ ]:
content = Puzzle(year=2020, day=4).input_data
content = [l.replace("\n", " ") for l in content.split('\n\n')]

In [ ]:
CODES = {"ecl", "pid", "eyr", "hcl", "byr", "iyr", "hgt"}

In [ ]:
solution = sum(map(lambda x: not(ALL-set(re.findall(r"(\w+):", x))), content))
#submit(solution, part="a", day=4, year=2020)

In [ ]:
solution = 0
for line in content:
    pairs = re.findall(r"(\w+):(\S+)", line)
    if CODES - set(p[0] for p in pairs):
        continue
        
    valid = True
    for k, v in pairs:
        if k == "byr":
            valid &= 1920 <= int(v) <= 2002
        
        elif k == "iyr":
            valid &= 2010 <= int(v) <= 2020
        
        elif k == "eyr":
            valid &= 2020 <= int(v) <= 2030
        
        elif k == "hgt":
            if v.endswith("cm"):
                valid &= 150 <= int(v[:-2]) <= 193
            elif v.endswith("in"):
                valid &= 59 <= int(v[:-2]) <= 76
        
        elif k == "hcl":
            valid &= bool(re.fullmatch(r'#[0-9a-f]{6}', v))
        
        elif k == "ecl":
            valid &= v in {"amb", "blu", "brn", "gry", "grn", "hzl", "oth"}
        
        elif k == "pid":
            valid &= bool(re.fullmatch(r'[0-9]{9}', v))
        
        elif k == "cid":
            valid &= True
            
    solution += valid
    
#submit(solution, part="b", day=4, year=2020)

## Day 5

http://adventofcode.com/2020/day/5

In [ ]:
content = Puzzle(year=2020, day=5).input_data.split()

In [ ]:
mapping = str.maketrans("FLBR", "0011")
IDS = sorted(int(line.translate(mapping), 2) for line in content)

In [ ]:
solution = IDS[-1]
#submit(solution, part="a", day=5, year=2020)

In [ ]:
solution = next(id2 - 1 for id1, id2 in zip(IDS, IDS[1:]) if id2 - id1 - 1)
#submit(solution, part="b", day=5, year=2020)

## Day 6

http://adventofcode.com/2020/day/6

In [ ]:
content = Puzzle(year=2020, day=6).input_data.split("\n\n")

In [ ]:
solution = sum(len(set.union(*[set(answer) for answer in answers.split()])) for answers in content)
#submit(solution, part="a", day=6, year=2020)

In [ ]:
solution = sum(len(set.intersection(*[set(answer) for answer in answers.split()])) for answers in content)
#submit(solution, part="b", day=6, year=2020)

## Day 7

http://adventofcode.com/2020/day/7

In [ ]:
content = Puzzle(year=2020, day=7).input_data.split("\n")

In [ ]:
mapping = collections.defaultdict(set)
mapping_inverse = collections.defaultdict(list)
for line in content:
    color = re.match(r"([\w ]+) bags contain", line)[1]
    for count, incolor in re.findall(r"(\d+) ([\w ]+) bag", line):
        mapping[incolor].add(color)
        mapping_inverse[color].append((int(count), incolor))

In [ ]:
acc = set()
def recurse(color):
    for c in mapping[color]:
        acc.add(c)
        recurse(c)
        
recurse("shiny gold")
solution = len(acc)
#submit(solution, part="a", day=7, year=2020)

In [ ]:
def cost(color):
    total = 0
    for count, incolor in mapping_inverse[color]:
        total += count
        total += count * cost(incolor)
    return total

solution = cost('shiny gold')
#submit(solution, part="b", day=7, year=2020)

## Day 8

http://adventofcode.com/2020/day/8

In [ ]:
content = Puzzle(year=2020, day=8).input_data.split("\n")
content = mapl(str.split, content)
content = mapl(lambda x: (x[0], int(x[1])), content)

In [ ]:
def run(prog, return_loop=False):
    acc = 0
    ptr = 0
    seen = set()
    while 0 <= ptr < len(prog):
        if ptr in seen:
            return acc if return_loop else None

        seen.add(ptr)

        inst, arg = prog[ptr]
        if inst == 'jmp':
            ptr += arg
            continue
        if inst == 'acc':
            acc += arg
        if inst == 'nop':
            pass

        ptr += 1
        
    return acc

solution = run(content, return_loop=True)
#submit(solution, part="a", day=8, year=2020)

In [ ]:
for idx, (inst, arg) in enumerate(content):
    prog = content[:]
    
    if inst == "jmp":
        prog[idx] = ("nop", arg)
    if inst == "nop":
        prog[idx] = ("jmp", arg)
    
    if inst in ["jmp", "nop"]:
        acc = run(prog)
        if acc:
            break

#submit(acc, part="b", day=8, year=2020)

## Day 9

http://adventofcode.com/2020/day/9

In [ ]:
content = Puzzle(year=2020, day=9).input_data
content